### Setup

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
!pip install cleverhans

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 10.1 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from datasets import load_from_disk
from transformers import CLIPTokenizer, CLIPProcessor, AutoTokenizer
from transformers import AutoProcessor, FlavaModel
from PIL import Image
import base64
import io
import numpy as np
import torch.nn.functional as F
import json

### PGD

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
"""The Fast Gradient Method attack."""
import numpy as np
import torch

from cleverhans.torch.utils import optimize_linear


def fast_gradient_method(
    model_fn,
    x,
    eps,
    norm,
    clip_min=None,
    clip_max=None,
    y=None,
    targeted=False,
    sanity_checks=False,
):
    """
    PyTorch implementation of the Fast Gradient Method.
    :param model_fn: a callable that takes an input tensor and returns the model logits.
    :param x: input tensor.
    :param eps: epsilon (input variation parameter); see https://arxiv.org/abs/1412.6572.
    :param norm: Order of the norm (mimics NumPy). Possible values: np.inf, 1 or 2.
    :param clip_min: (optional) float. Minimum float value for adversarial example components.
    :param clip_max: (optional) float. Maximum float value for adversarial example components.
    :param y: (optional) Tensor with true labels. If targeted is true, then provide the
              target label. Otherwise, only provide this parameter if you'd like to use true
              labels when crafting adversarial samples. Otherwise, model predictions are used
              as labels to avoid the "label leaking" effect (explained in this paper:
              https://arxiv.org/abs/1611.01236). Default is None.
    :param targeted: (optional) bool. Is the attack targeted or untargeted?
              Untargeted, the default, will try to make the label incorrect.
              Targeted will instead try to move in the direction of being more like y.
    :param sanity_checks: bool, if True, include asserts (Turn them off to use less runtime /
              memory or for unit tests that intentionally pass strange input)
    :return: a tensor for the adversarial example
    """
    if norm not in [np.inf, 1, 2]:
        raise ValueError(
            "Norm order must be either np.inf, 1, or 2, got {} instead.".format(norm)
        )
    if eps < 0:
        raise ValueError(
            "eps must be greater than or equal to 0, got {} instead".format(eps)
        )
    if eps == 0:
        return x
    if clip_min is not None and clip_max is not None:
        if clip_min > clip_max:
            raise ValueError(
                "clip_min must be less than or equal to clip_max, got clip_min={} and clip_max={}".format(
                    clip_min, clip_max
                )
            )

    asserts = []

    # If a data range was specified, check that the input was in that range
    """
    if clip_min is not None:
        assert_ge = torch.all(
            torch.ge(x, torch.tensor(clip_min, device=x.device, dtype=x.dtype))
        )
        asserts.append(assert_ge)

    if clip_max is not None:
        assert_le = torch.all(
            torch.le(x, torch.tensor(clip_max, device=x.device, dtype=x.dtype))
        )
        asserts.append(assert_le)
    """

    # x needs to be a leaf variable, of floating point type and have requires_grad being True for
    # its grad to be computed and stored properly in a backward call

    x['pixel_values'] = x['pixel_values'].detach().clone().to(torch.float).requires_grad_(True)

    out = model_fn(**x)

    if y is None:
        # Using model predictions as ground truth to avoid label leaking
        y = out.logits_per_image.argmax(dim=1)

    # Compute loss
    loss_fn = torch.nn.CrossEntropyLoss()
    loss = loss_fn(out.logits_per_image.softmax(dim=1), y)
    # If attack is targeted, minimize loss of target label rather than maximize loss of correct label
    if targeted:
        loss = -loss

    # Define gradient of loss wrt input
    loss.backward()
    optimal_perturbation = optimize_linear(x['pixel_values'].grad, eps, norm)

    # Add perturbation to original example to obtain adversarial example
    adv_x = copy.deepcopy(x)
    adv_x['pixel_values'] = x['pixel_values'] + optimal_perturbation

    # If clipping is needed, reset all values outside of [clip_min, clip_max]
    """
    if (clip_min is not None) or (clip_max is not None):
        if clip_min is None or clip_max is None:
            raise ValueError(
                "One of clip_min and clip_max is None but we don't currently support one-sided clipping"
            )
        adv_x['pixel_values'] = torch.clamp(adv_x['pixel_values'], clip_min, clip_max)

    if sanity_checks:
        assert np.all(asserts)
    """
    return adv_x, loss

"""The Projected Gradient Descent attack."""
import numpy as np
import torch
from torchvision import transforms
#from cleverhans.torch.attacks.CLIP.fast_gradient_method import fast_gradient_method
#from cleverhans.torch.utils import clip_eta

import copy
import torch.nn.functional as F

torch.autograd.set_detect_anomaly(True)

def clip_eta(eta, norm, eps):
    """
    PyTorch implementation of the clip_eta in utils_tf.

    :param eta: Tensor
    :param norm: np.inf, 1, or 2
    :param eps: float
    """
    if norm not in [np.inf, 1, 2]:
        raise ValueError("norm must be np.inf, 1, or 2.")
    eta = eta.clone()
    avoid_zero_div = torch.tensor(1e-12, dtype=eta.dtype, device=eta.device)
    reduc_ind = list(range(1, len(eta.size())))
    if norm == np.inf:
        eta = torch.clamp(eta, -eps, eps)
    else:
        if norm == 1:
            raise NotImplementedError("L1 clip is not implemented.")
            norm = torch.max(
                avoid_zero_div, torch.sum(torch.abs(eta), dim=reduc_ind, keepdim=True)
            )
        elif norm == 2:
            norm = torch.sqrt(
                torch.max(
                    avoid_zero_div, torch.sum(eta ** 2, dim=reduc_ind, keepdim=True)
                )
            )
        factor = torch.min(
            torch.tensor(1.0, dtype=eta.dtype, device=eta.device), eps / norm
        )
        eta *= factor
    return eta

def projected_gradient_descent(
    model_fn,
    x,
    eps,
    eps_iter,
    nb_iter,
    norm,
    clip_min=None,
    clip_max=None,
    y=None,
    ori_x=None,
    time=None,
    targeted=False,
    rand_init=True,
    rand_minmax=None,
    sanity_checks=True,
    method=None,
    model=None
):
    """
    This class implements either the Basic Iterative Method
    (Kurakin et al. 2016) when rand_init is set to False. or the
    Madry et al. (2017) method if rand_init is set to True.
    Paper link (Kurakin et al. 2016): https://arxiv.org/pdf/1607.02533.pdf
    Paper link (Madry et al. 2017): https://arxiv.org/pdf/1706.06083.pdf
    :param model_fn: a callable that takes an input tensor and returns the model logits.
    :param x: input tensor.
    :param eps: epsilon (input variation parameter); see https://arxiv.org/abs/1412.6572.
    :param eps_iter: step size for each attack iteration
    :param nb_iter: Number of attack iterations.
    :param norm: Order of the norm (mimics NumPy). Possible values: np.inf, 1 or 2.
    :param clip_min: (optional) float. Minimum float value for adversarial example components.
    :param clip_max: (optional) float. Maximum float value for adversarial example components.
    :param y: (optional) Tensor with true labels. If targeted is true, then provide the
              target label. Otherwise, only provide this parameter if you'd like to use true
              labels when crafting adversarial samples. Otherwise, model predictions are used
              as labels to avoid the "label leaking" effect (explained in this paper:
              https://arxiv.org/abs/1611.01236). Default is None.
    :param targeted: (optional) bool. Is the attack targeted or untargeted?
              Untargeted, the default, will try to make the label incorrect.
              Targeted will instead try to move in the direction of being more like y.
    :param rand_init: (optional) bool. Whether to start the attack from a randomly perturbed x.
    :param rand_minmax: (optional) bool. Support of the continuous uniform distribution from
              which the random perturbation on x was drawn. Effective only when rand_init is
              True. Default equals to eps.
    :param sanity_checks: bool, if True, include asserts (Turn them off to use less runtime /
              memory or for unit tests that intentionally pass strange input)
    :return: a tensor for the adversarial example
    """
    """
    if method is None or model is None:
        raise ValueError("method and model are None.")
    """
    if norm == 1:
        raise NotImplementedError(
            "It's not clear that FGM is a good inner loop"
            " step for PGD when norm=1, because norm=1 FGM "
            " changes only one pixel at a time. We need "
            " to rigorously test a strong norm=1 PGD "
            "before enabling this feature."
        )
    if norm not in [np.inf, 2]:
        raise ValueError("Norm order must be either np.inf or 2.")
    if eps < 0:
        raise ValueError(
            "eps must be greater than or equal to 0, got {} instead".format(eps)
        )
    if eps == 0:
        return x
    if eps_iter < 0:
        raise ValueError(
            "eps_iter must be greater than or equal to 0, got {} instead".format(
                eps_iter
            )
        )
    if eps_iter == 0:
        return x

    assert eps_iter <= eps, (eps_iter, eps)
    if clip_min is not None and clip_max is not None:
        if clip_min > clip_max:
            raise ValueError(
                "clip_min must be less than or equal to clip_max, got clip_min={} and clip_max={}".format(
                    clip_min, clip_max
                )
            )

    asserts = []
    """
    # If a data range was specified, check that the input was in that range
    if clip_min is not None:
        assert_ge = torch.all(
            torch.ge(x, torch.tensor(clip_min, device=x.device, dtype=x.dtype))
        )
        asserts.append(assert_ge)

    if clip_max is not None:
        assert_le = torch.all(
            torch.le(x, torch.tensor(clip_max, device=x.device, dtype=x.dtype))
        )
        asserts.append(assert_le)
    """
    adv_x = copy.deepcopy(x)
    ori_x = x

    # Initialize loop variables
    if time==0:
        rand_init=True
    else:
        rand_init=False
    # print(rand_init)rand _init
    if rand_init:
        if rand_minmax is None:
            rand_minmax = eps
        eta = torch.zeros_like(ori_x['pixel_values']).uniform_(-rand_minmax, rand_minmax)

    else:
        eta = torch.zeros_like(ori_x['pixel_values'])

    # Clip eta
    eta = clip_eta(eta, norm, eps)
    #eta=eta.requires_grad_(True)
    # if time==0:
    #     print('eta', eta[0,:10,0,0])
    adv_x['pixel_values'] = ori_x['pixel_values'] + eta
    if clip_min is not None or clip_max is not None:
        adv_x['pixel_values'] = torch.clamp(adv_x['pixel_values'], clip_min, clip_max)
    """
    if y is None:
        # Using model predictions as ground truth to avoid label leaking
        _, y = torch.max(model_fn(x), 1)
    """
    i = 0
    loss_list=[]
    # loss_last=loss_ini
    # step=0
    ori_x['pixel_values']=ori_x['pixel_values'].requires_grad_(False)
    while i < nb_iter:
        #print(f"=========Iteration {i}===========")
        adv_x,loss = fast_gradient_method(
            model_fn,
            adv_x,
            eps_iter,
            norm,
            clip_min=clip_min,
            clip_max=clip_max,
            y=y,
            targeted=targeted
        )
        loss_list.append(float(loss.detach().cpu().numpy()))
        eta = adv_x['pixel_values'] - ori_x['pixel_values']
        #eta = eta.clone()

        # print(torch.max(eta), torch.min(eta))
        #clipped_eta = clip_eta(eta, norm, eps)
        adv_x['pixel_values'] = ori_x['pixel_values'] + clip_eta(eta, norm, eps)
        if clip_min is not None or clip_max is not None:
            adv_x['pixel_values'] = torch.clamp(adv_x['pixel_values'], clip_min, clip_max)
        i += 1
        #print(adv_x['pixel_values'])

        logits = model_fn(**adv_x).logits_per_image
        #print(logits)

        #preds = model_fn(**adv_x).logits_per_image.softmax(dim=1)
        #print(preds)

    asserts.append(eps_iter <= eps)
    if norm == np.inf and clip_min is not None:
        # TODO necessary to cast clip_min and clip_max to x.dtype?
        asserts.append(eps + clip_min <= clip_max)
    # print(asserts)
    asserts=[i.cpu() for i in asserts if i is not True]
    # exit()
    if sanity_checks:
        assert np.all(asserts)
    return adv_x,loss_list

In [ ]:
# Load the trained model
#pretrained_model  ='openai/clip-vit-large-patch14'
pretrained_model  ='openai/clip-vit-base-patch32'

image_processor = CLIPProcessor.from_pretrained(pretrained_model)
text_processor = CLIPTokenizer.from_pretrained(pretrained_model)
model = CLIPModel.from_pretrained(pretrained_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [ ]:
batch_size = 1
from datasets import load_from_disk

combined = load_from_disk('./processed_data/combined_hateful_memes_dataset')
dev_unseen_data = combined['dev_unseen']

In [ ]:
dev_unseen_dataset = ImageCaptioningDataset(dev_unseen_data)
dev_unseen_loader = DataLoader(dev_unseen_dataset, shuffle=True, batch_size=batch_size)

In [ ]:
import matplotlib.pyplot as plt
from torchvision.transforms import ToPILImage
import json

results = {}

def tensor_to_PIL(img_tensor):
    # rescale range
    img_tensor -= img_tensor.min()
    img_tensor /= img_tensor.max()
    img = ToPILImage()(img_tensor)
    img = img.resize((224, 224))
    #img.show()
    return img

for batch_idx, batch in enumerate(dev_unseen_loader):

    meme_idx = batch['idx_memes'][0]
    print(f"{batch_idx}: Processing image {meme_idx}")
    features = {key: batch[key].squeeze(0) for key in ['pixel_values', 'input_ids', 'attention_mask']}

    out = model(**features)

    results[meme_idx] = {"label": batch['labels'],
                         "ori_preds": out.logits_per_image.softmax(dim=1)}

    adv_features = projected_gradient_descent(model,
                            features,
                            eps=0.2,
                            eps_iter=0.1,
                            nb_iter=5,
                            norm=np.inf,
                            clip_min=batch['pixel_values'].squeeze(0).min(),
                            clip_max=batch['pixel_values'].squeeze(0).max(),
                            y=batch['labels'].squeeze(0))[0]
    # plot images
    adv_out = model(**adv_features)
    adv_preds = adv_out.logits_per_image.softmax(dim=1)

    results[meme_idx].update({"adv_preds": adv_preds})
    print(batch['labels'])
    print(adv_preds)
    adv_img = tensor_to_PIL(adv_features['pixel_values'][0])
    adv_img.save(f'hateful_memes/adv_img/adv_{meme_idx}.png')

{'input_ids': tensor([[49406,   783, 36418, 49407],
        [49406, 36418, 49407, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1],
        [1, 1, 1, 0]])}
0: Processing image 58679
tensor([[0]])
tensor([[0.0069, 0.9931]], grad_fn=<SoftmaxBackward0>)
{'input_ids': tensor([[49406,   783, 36418, 49407],
        [49406, 36418, 49407, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1],
        [1, 1, 1, 0]])}
1: Processing image 76348
tensor([[1]])
tensor([[0.9979, 0.0021]], grad_fn=<SoftmaxBackward0>)
{'input_ids': tensor([[49406,   783, 36418, 49407],
        [49406, 36418, 49407, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1],
        [1, 1, 1, 0]])}
2: Processing image 85271
tensor([[0]])
tensor([[7.2464e-04, 9.9928e-01]], grad_fn=<SoftmaxBackward0>)
{'input_ids': tensor([[49406,   783, 36418, 49407],
        [49406, 36418, 49407, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1],
        [1, 1, 1, 0]])}
3: Processing image 85209
tensor([[0]])
tensor([[0.0112, 0.9888]], grad_fn=<Softmax

KeyboardInterrupt: 

In [ ]:
for k, v in results.items():
    for k1,v1 in v.items():
        if isinstance(v1, torch.Tensor):
            results[k][k1] = v1.cpu().data.numpy().tolist()
        if isinstance(v1, np.ndarray):
            results[k][k1] = v1.tolist()

In [ ]:
with open("adv_dev_unseen_results.json", "w") as f:
    json.dump(results, f)